In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()
data = spark.read.csv("content_watch2.csv",sep = '\t', inferSchema=True, header=True).cache()
type(data)
data.dtypes
data.schema
data.describe().show()
data.count()

+-------+--------------------+--------------------+------------------+-----------------+------------------+
|summary|             user_id|       operator_name|        content_id|    content_title|     show_duration|
+-------+--------------------+--------------------+------------------+-----------------+------------------+
|  count|               20000|               20000|             20000|            20000|             20000|
|   mean|   8.6525109116155E8|                null|        17391.6428|1289.111111111111|1272.5749665327978|
| stddev|1.6811632041240516E8|                null|32636.639667408333| 958.215326067743|1294.6419975352087|
|    min|               46429|21 Century Teleco...|               949|     #SELFIEPARTY|                 0|
|    max|          1010558449|     umos Center LLC|            191504|Я – телохранитель|              None|
+-------+--------------------+--------------------+------------------+-----------------+------------------+



20000

In [11]:
data = data.na.replace('None', '0', 'show_duration') # заменяем None на 0 

In [12]:
data.describe().show()

+-------+--------------------+--------------------+------------------+-----------------+-----------------+
|summary|             user_id|       operator_name|        content_id|    content_title|    show_duration|
+-------+--------------------+--------------------+------------------+-----------------+-----------------+
|  count|               20000|               20000|             20000|            20000|            20000|
|   mean|   8.6525109116155E8|                null|        17391.6428|1289.111111111111|        1140.7362|
| stddev|1.6811632041240516E8|                null|32636.639667408333| 958.215326067743|1285.630681853096|
|    min|               46429|21 Century Teleco...|               949|     #SELFIEPARTY|                0|
|    max|          1010558449|     umos Center LLC|            191504|Я – телохранитель|              997|
+-------+--------------------+--------------------+------------------+-----------------+-----------------+



**Какой фильм был просмотрен максимальным количеством человек?
Какой пользователь посмотрел больше всех фильмов? **

In [13]:
data[data.show_duration != '0'].groupBy('content_title')\
    .agg(countDistinct('user_id').alias('count')).orderBy('count', ascending = False).head(1)

[Row(content_title='Мастер и Маргарита', count=1586)]

In [14]:
data.groupBy('user_id').agg(countDistinct('content_title').alias('count'))\
    .orderBy('count', ascending = False).head(1)

[Row(user_id=446413452, count=6)]

** У какого оператора больше всего пользователей? **

In [15]:
data.groupBy('operator_name').agg(countDistinct('user_id').alias('count'))\
    .orderBy('count', ascending = False).head(1)

[Row(operator_name='Rostelecom', count=2232)]

** Какой средний показатель по числу просмотренных фильмов среди мужчин - пользователей Ростелекома? **

In [23]:
data = data.withColumn('rand_num', rand())
data = data.withColumn('gender', when(data['rand_num'] >= 0.4, 'male').otherwise('female')).drop('rand_num')
data.show()

+----------+--------------------+----------+--------------------+-------------+------+
|   user_id|       operator_name|content_id|       content_title|show_duration|gender|
+----------+--------------------+----------+--------------------+-------------+------+
| 827375963|          Rostelecom|      9966|                Луна|         1400|  male|
| 890810589|          LLC TOMTEL|     10140|             Солдаты|         1271|  male|
| 929088258|OJS Moscow city t...|      9572|               Мажор|           30|  male|
| 987577016|          Rostelecom|      8014|Клуб Винкс – Школ...|         1310|  male|
| 573841374|OJS Moscow city t...|      9983|               Метод|         3205|female|
| 757852413|Ekaterinburg-2000...|      9572|               Мажор|         3105|female|
| 991611208|RECONN. Operator ...|      9983|               Метод|         2998|  male|
| 914647005|          Rostelecom|      7097|      Закрытая школа|         2870|  male|
| 924157490|Multiservice Netw...|     10073

In [24]:
data[(data.gender == 'male') & (data.operator_name == 'Rostelecom')]\
    .groupBy('user_id').agg(countDistinct('content_title').alias('count'))\
    .agg({'count': 'mean'}).collect()

[Row(avg(count)=1.0210656753407683)]

** Построить по данным файла content_watch.csv матрицу user-item **

In [30]:
user_item = data.groupBy('user_id').agg(collect_set('content_id'))
user_item.show()

+---------+-----------------------+
|  user_id|collect_set(content_id)|
+---------+-----------------------+
|434539289|                [10209]|
|471496565|                 [6859]|
|512335285|                 [9570]|
|584666375|                 [9478]|
|603215036|                [10209]|
|603694929|                 [7498]|
|643879096|                 [9983]|
|732456832|                [10121]|
|732863641|                 [8190]|
|761100659|                 [9108]|
|803481291|                 [7121]|
|804722319|                 [7651]|
|808859842|                 [9399]|
|826601258|                 [6859]|
|828890717|                 [7312]|
|829340245|                 [7607]|
|854955816|                 [7305]|
|855900980|                 [9547]|
|857695481|                 [7305]|
|866622249|                 [7607]|
+---------+-----------------------+
only showing top 20 rows

